In [1]:
import os
import base64

In [2]:
with open("../data/secrets_itor.json") as secrets:
    secrets_dict = eval(secrets.read())
    open_api_key = base64.b64decode(secrets_dict["openai_api_key"]).decode('ascii')
    os.environ["OPENAI_API_KEY"] = open_api_key
    if "organization_id" in secrets_dict.keys():
        openai_organization = base64.b64decode(secrets_dict["organization_id"]).decode('ascii')
        os.environ["OPENAI_ORGANIZATION"] = openai_organization

In [3]:
import openai

In [4]:
with open('../data/directiva_residuos/mods_2018.json') as file:
    mods_2018 = eval(file.read())
with open('../data/directiva_residuos/articles_2008.json') as file:
    articles_2008 = eval(file.read())

# Detect article to modif

In [5]:
detect_article_modif_system_prompt = """
Eres un asistente que identifica en un texto legal los artículos que modifica de otra legislación anterior.
La respuesta debe ser únicamente los nombre de los artículos identificados en minúscula, sin tildes y con barra baja en lugar de espacio.
Los artículos identificados deben estar estar formateados como una lista de python ["articulo_1", "articulo_2"].

Ejemplo 1
Texto de modificaciones(encapsulado en triple tilde inversa):
```
El artículo 1 se sustituye por el texto siguiente:
Exposición de motivos y causas.
Lorem ipsum dolor sit amet, consectetur adipiscing elit.
```
Artículos identificados:
["articulo_1"]

Ejemplo 2
Texto de modificaciones(encapsulado en triple tilde inversa):
```
En el artículo 2, apartado 2, se añade la letra siguiente:
Etiam mattis mauris nec felis tempus dapibus.

3) El artículo 3 se modifica como sigue:
Proin pretium, est sed pellentesque rutrum, ipsum felis posuere dui, quis pulvinar nibh neque quis libero.
```
Artículos identificados:
["articulo_1", "articulo_3"]
"""

In [6]:
modif_doc_prompt_prompt = """

Texto de modificaciones (encapsulado en triple tilde inversa):
```
{mod_doc}
```

Artículos identificados:
"""

In [11]:
params = {
#   'model': 'gpt-4',
  'model': 'gpt-3.5-turbo-16k',
  'request_timeout': None,
  'max_tokens': 2000,
  'stream': False,
  'n': 1,
  'temperature': 0,
  'top_p': 1,
  'api_base': '',
  'organization': '',
}

In [9]:
message_dicts = [{'role': 'system',
   'content': detect_article_modif_system_prompt},
  {'role': 'user',
   'content': modif_doc_prompt_prompt.format(mod_doc=mods_2018["1"])},
]

In [7]:
print(mods_2018["1"])

2)
En el artículo 2, apartado 2, se añade la letra siguiente:
«e)
sustancias que se destinen a ser utilizadas como materias primas para piensos tal como se definen en el artículo 3, apartado 2, letra g), del
Reglamento (CE) n.o 767/2009 del Parlamento Europeo y del Consejo (*1) y que no sean subproductos animales ni los contengan;
(*1) Reglamento (CE) n.o 767/2009 del Parlamento Europeo y del
Consejo, de 13 de julio de 2009, sobre la comercialización y la utilización de los piensos, por el que se modifica el Reglamento (CE) n.o 1831/2003
y se derogan las Directivas 79/373/CEE del Consejo, 80/511/CEE de la
Comisión, 82/471/CEE del Consejo, 83/228/CEE del Consejo, 93/74/CEE
del Consejo, 93/113/CE del Consejo y 96/25/CE del Consejo y la Decisión
2004/217/CE de la Comisión (DO L 229 de 1.9.2009, p. 1).»."


In [12]:
response = openai.ChatCompletion.create(
    messages=message_dicts,
    **params
)

In [100]:
print(response.choices[0].message.content)

["articulo_2", "articulo_3"]


In [36]:
isinstance(eval(response.choices[0].message.content), list)

True

In [13]:
def identify_article(modification_text: str, api_params: dict):
    message_dicts = [
        {"role": "system", "content": detect_article_modif_system_prompt},
        {"role": "user", "content": modif_doc_prompt_prompt.format(mod_doc=modification_text)},
    ]
    response_text = openai.ChatCompletion.create(messages=message_dicts, **api_params).choices[0].message.content
    response = eval(response_text)
    assert isinstance(response, list), "Wrong output format:\n{response_text}"
    return response

In [106]:
identify_article(mods_2018["1"], api_params=params)

['articulo_2', 'articulo_3']

# Diff Generator

In [14]:
diff_system_prompt = """
Eres un experto asistente legal que ayuda a interpretar y entender cambios en legislaciones. Tienes experiencia redactando, corrigiendo y mejorando textos legales como leyes o directivas.
Tu objetivo es consolidar los cambios en los documentos tachando (usa el tag html <del>) la parte del texto que queda modificada o derogada y añadiendo el nuevo texto  (usa el tag html <ins>). Pueden haber solo eliminaciones o solo adiciones de texto.
El resultado final será un texto único dónde quede claro cuales son las partes del texto que quedan obsoletas y aquellas que entran en vigor.
Deber realizar el trabajo artículo a artículo para evitar errores.
Si las instrucciones de modificación no afectan a los artículos en el documento original, o se trata de otro elemento que no sea un artículo, simplemente responde con un string vacío.

Ejemplo:

Documento original (encapsulado en triple tilde inversa):
```
Constitución española
Artículo 1.
1. España se constituye en un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico la libertad, la justicia, la igualdad y el pluralismo político. 
2. La soberanía nacional reside en el pueblo español, del que emanan los poderes del Estado. 
3. La forma política del Estado español es la Monarquía parlamentaria. 

Artículo 2.
La Constitución se fundamenta en la indisoluble unidad de la Nación española, patria común e indivisible de todos los españoles, y reconoce y garantiza el derecho a la autonomía de las nacionalidades y regiones que la integran y la solidaridad entre todas ellas.

Artículo 3.
1. El castellano es la lengua española oficial del Estado. Todos los españoles tienen el deber de conocerla y el derecho a usarla. 
2. Las demás lenguas españolas serán también oficiales en las respectivas Comunidades Autónomas de acuerdo con sus Estatutos. 
3. La riqueza de las distintas modalidades lingüísticas de España es un patrimonio cultural que será objeto de especial respeto y protección.
```

Instrucciones de modificaciones (encapsulado en triple tilde inversa):
```
Constitución Española
Del artículo 1 se modifican los siguientes apartados: 
Apartado 1. Se sustituye el texto "la libertad, la justicia, la igualdad y el pluralismo político." por "la unidad, el respeto y el amor."
Apartado 3. Se sustituye por completo por el siguiente texto "La forma política de España es la República parlamentaria."

En el artículo 2 se añade el siguiente texto "Sin menoscabo de su derecho a decidir sobre su futuro.".

En el articulo 3 se elimina el apartado 3.
```

Documento consolidado (encapsulado en triple tilde inversa):
```
Artículo 1.
1. España se constituye en un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico ≤del>la libertad, la justicia, la igualdad y el pluralismo político. </del> la unidad, el respeto y el amor.
2. La soberanía nacional reside en el pueblo español, del que emanan los poderes del Estado. 
3. <del>La forma política del Estado español es la  Monarquía parlamentaria</del> <ins>La forma política de España es la República parlamentaria.</ins>

Artículo 2.
La Constitución se fundamenta en la indisoluble unidad de la Nación española, patria común e indivisible de todos los españoles, y reconoce y garantiza el derecho a la autonomía de las nacionalidades y regiones que la integran y la solidaridad entre todas ellas. <ins>Sin menoscabo de su derecho a decidir sobre su futuro.</ins>

Artículo 3.
1. El castellano es la lengua española oficial del Estado. Todos los españoles tienen el deber de conocerla y el derecho a usarla. 
2. Las demás lenguas españolas serán también oficiales en las respectivas Comunidades Autónomas de acuerdo con sus Estatutos. 
<del>3. La riqueza de las distintas modalidades lingüísticas de España es un patrimonio cultural que será objeto de especial respeto y protección.</del>
```
"""

In [15]:
diff_gen_prompt = """
Documento original (encapsulado en triple tilde inversa):
```
{orig_doc}
```

Instrucciones de modificaciones (encapsulado en triple tilde inversa):
```
{mod_doc}
```

Documento consolidado:
"""

In [112]:
message_dicts = [
    {"role": "system", "content": diff_system_prompt},
    {
        "role": "user",
        "content": diff_gen_prompt.format(orig_doc=articles_2008["articulo_1"], mod_doc=mods_2018["0"]),
    },
]

In [121]:
params = {
  'model': 'gpt-4',
#   'model': 'gpt-3.5-turbo-16k',
  'request_timeout': None,
  'max_tokens': 2000,
  'stream': False,
  'n': 1,
  'temperature': 0,
  'top_p': 1,
  'api_base': '',
  'organization': '',}

In [114]:
response = openai.ChatCompletion.create(
    messages=message_dicts,
    **params
)

In [115]:
print(response.choices[0].message.content)

Artículo 1
Objeto y ámbito de aplicación
La presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de la generación de residuos y de los impactos negativos de la generación y gestión de los residuos, mediante la reducción del impacto global del uso de los recursos y mediante la mejora de la eficiencia de dicho uso, elementos cruciales para efectuar la transición a una economía circular y garantizar la competitividad de la Unión a largo plazo.


In [16]:
def generate_diff_text(origin_text: str, mod_text: str, api_params: dict):
    message_dicts = [
        {"role": "system", "content": diff_system_prompt},
        {"role": "user", "content": diff_gen_prompt.format(orig_doc=origin_text, mod_doc=mod_text)},
    ]
    response_text = openai.ChatCompletion.create(messages=message_dicts, **api_params).choices[0].message.content
    return response_text

In [122]:
response = generate_diff_text(origin_text=articles_2008["articulo_3"], mod_text=mods_2018["1"], api_params=params)

In [123]:
print(response)

""


# Full pipeline

In [17]:
def auto_generate_diff_text(
    origin_docs: dict, mod_docs: dict, diff_api_params: dict, id_article_api_params: dict, verbose: bool = False
):
    diffed_docs = {}
    for mod_key, mod_doc in mod_docs.items():
        articles = identify_article(mod_doc, api_params=id_article_api_params)
        for article in articles:
            if verbose:
                print(mod_key, article)
            res = generate_diff_text(origin_text=origin_docs[article], mod_text=mod_doc, api_params=diff_api_params)
            if not res and verbose:
                print(f"Article {article} not affected by modification {mod_key}")
            diffed_docs.update({article: res})
    return diffed_docs

In [18]:
diff_api_params = {
  'model': 'gpt-4',
#   'model': 'gpt-3.5-turbo-16k',
  'request_timeout': None,
  'max_tokens': 2000,
  'stream': False,
  'n': 1,
  'temperature': 0,
  'top_p': 1,
  'api_base': '',
  'organization': '',}

In [19]:
id_article_api_params = {
#   'model': 'gpt-4',
  'model': 'gpt-3.5-turbo-16k',
  'request_timeout': None,
  'max_tokens': 2000,
  'stream': False,
  'n': 1,
  'temperature': 0,
  'top_p': 1,
  'api_base': '',
  'organization': '',}

In [20]:
mod_2018_subset = {k:v for k,v in mods_2018.items() if k in ("0", "1")}

In [21]:
response_test = auto_generate_diff_text(
    origin_docs=articles_2008,
    mod_docs=mod_2018_subset,
    diff_api_params=diff_api_params,
    id_article_api_params=id_article_api_params,
    verbose=True,
)

0 articulo_1
1 articulo_2
1 articulo_3


In [22]:
response_test

{'articulo_1': '```\nArtículo 1\nObjeto y ámbito de aplicación\n<del>La presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de los impactos adversos de la generación y gestión de los residuos, la reducción de los impactos globales del uso de los recursos y la mejora de la eficacia de dicho uso.</del> <ins>La presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de la generación de residuos y de los impactos negativos de la generación y gestión de los residuos, mediante la reducción del impacto global del uso de los recursos y mediante la mejora de la eficiencia de dicho uso, elementos cruciales para efectuar la transición a una economía circular y garantizar la competitividad de la Unión a largo plazo.</ins>\n```',
 'articulo_2': '```\nArtículo 2\nExclusiones del ámbito de aplicación\n1.\nQueda excluido lo siguiente del ámbito d

In [26]:
print(mods_2018["1"])

2)
En el artículo 2, apartado 2, se añade la letra siguiente:
«e)
sustancias que se destinen a ser utilizadas como materias primas para piensos tal como se definen en el artículo 3, apartado 2, letra g), del
Reglamento (CE) n.o 767/2009 del Parlamento Europeo y del Consejo (*1) y que no sean subproductos animales ni los contengan;
(*1) Reglamento (CE) n.o 767/2009 del Parlamento Europeo y del
Consejo, de 13 de julio de 2009, sobre la comercialización y la utilización de los piensos, por el que se modifica el Reglamento (CE) n.o 1831/2003
y se derogan las Directivas 79/373/CEE del Consejo, 80/511/CEE de la
Comisión, 82/471/CEE del Consejo, 83/228/CEE del Consejo, 93/74/CEE
del Consejo, 93/113/CE del Consejo y 96/25/CE del Consejo y la Decisión
2004/217/CE de la Comisión (DO L 229 de 1.9.2009, p. 1).»."


In [24]:
print(response_test["articulo_2"])

```
Artículo 2
Exclusiones del ámbito de aplicación
1.
Queda excluido lo siguiente del ámbito de aplicación de la presente
Directiva:
a) los efluentes gaseosos emitidos en la atmósfera;
b) la tierra (in situ) incluido el suelo no excavado contaminado y los edificios en contacto permanente con la tierra;
c) suelo no contaminado y demás material en estado natural excavado durante las actividades de construcción cuando se tiene la certeza de que el material se utilizará a efectos de construcción en su estado natural en el sitio del que se extrajo;
d) los residuos radiactivos;
e) explosivos desclasificados;
f) materias fecales, si no están contempladas en el apartado 2.b), paja y otro material natural, agrícola o silvícola, no peligroso, utilizado en la agricultura, en la silvicultura o en la producción de energía a base de esta biomasa, mediante procedimientos o métodos que no dañen el medio ambiente o pongan en peligro la salud humana.
2.
Queda excluido lo siguiente del ámbito de aplicac

### Further work needed:
- Obtener el número de tokens de cada artículo y modificación mediante tiktoke. Así se podrá hacer una estimación rápida de cuantos caben en una llamada a GPT-4 y aprovechar llamadas. Tener en cuenta que los system prompts repetidos "desperdician" ese gasto de tokens, por eso es útil unir requests.
- Testear gpt3.5 vs gpt4 para prompts largos
- Quitar los pie de página e.g. print(mods_2018["1"])